# ⚡ XGBoost (Extreme Gradient Boosting)

XGBoost는 **Gradient Boosting** 알고리즘을 고도화한 라이브러리로,  
빠른 속도, 높은 성능, 과적합 제어 기능을 제공하는 **트리 기반 앙상블 학습기**입니다.  
대규모 데이터와 Kaggle 대회에서 자주 사용되는 강력한 모델입니다.

---

## 1) 주요 클래스
- **xgboost.XGBClassifier** : 분류용
- **xgboost.XGBRegressor** : 회귀용

---

## 2) 주요 매개변수 (Parameters, 기본값 포함)

## 📌 1. 학습 제어 관련
- **n_estimators (기본=100)**  
  - 부스팅 단계(트리) 개수. 많을수록 성능↑, 계산량↑, 과적합 위험↑.  
  - `learning_rate`와 trade-off 관계.

- **learning_rate (eta, 기본=0.1)**  
  - 단계별 학습률(수축 계수).  
  - 값↓ → 일반화↑, 더 많은 트리 필요. (예: 0.01~0.3 권장)  
  - 보통 `n_estimators`와 함께 Grid/Random Search로 조정.
- **early_stopping_rounds** : int, optional  
  - `eval_set`에서 지정된 평가 지표가 **n 라운드 연속 개선되지 않으면 학습 중단**  
  - `n_estimators`는 크게 잡고, 이 값으로 최적 라운드 탐색  
- **objective**  
  - 손실 함수 지정.  
  - 분류: 
    - **"binary:logistic"**  
      - 이진 분류용 (0/1).  
      - 출력: 확률 (로지스틱 시그모이드 적용).  
      - `predict_proba`와 연결.

    - **"binary:logitraw"**  
      - 이진 분류용.  
      - 출력: 로짓(log-odds) 값 (확률 아님).  
      - 확률 후처리 직접 하고 싶을 때 사용.

    - **"multi:softmax"**  
      - 다중 클래스 분류.  
      - 출력: 클래스 인덱스 (정수).  

    - **"multi:softprob"**  
      - 다중 클래스 분류.  
      - 출력: 각 클래스별 확률 벡터.  
      - 확률 기반 후처리/평가에 적합.
  - 회귀: 
    - **"reg:squarederror"** (기본)  
      - 평균 제곱 오차(MSE) 기반.  
      - 일반적인 연속형 타깃 예측에 사용.

    - **"reg:absoluteerror"**  
      - 평균 절대 오차(MAE).  
      - 이상치(outlier)에 더 강건.

    - **"reg:squaredlogerror"**  
      - 로그 스케일에서의 MSE.  
      - 타깃 값이 양수이고 값의 범위가 넓을 때 사용 (예: 가격, 소득).

    - **"reg:pseudohubererror"**  
      - pseudo-Huber 손실.  
      - MSE와 MAE의 절충 → 이상치에 완화된 영향.

- **eval_metric**  
  - 검증 시 사용할 평가 지표.  
  - 분류: `"logloss"`, `"auc"`, `"error"`  
  - 회귀: `"rmse"`, `"mae"` 등  
  - 조기 종료(early stopping)와 함께 사용.

---

## 📌 2. 트리 구조 제어
- **max_depth (기본=6)**  
  - 트리 최대 깊이. 깊으면 복잡도↑, 과적합 위험↑.  
  - 보통 3~10 범위 탐색.

- **min_child_weight (기본=1)**  
  - 리프 노드가 분할되기 위한 최소 가중치 합(Hessian).  
  - 값↑ → 보수적 분할(과적합↓), 값↓ → 더 세밀한 분할 허용.

- **gamma (기본=0)**  
  - 리프 추가 분할에 필요한 최소 손실 감소.  
  - 값↑ → 불필요한 분할 억제 → 과적합 방지.

- **max_leaves**  
  - 리프 수 제한. `grow_policy="lossguide"`와 함께 사용.  
  - 깊이 대신 리프 개수로 복잡도 제어 가능.

---

## 📌 3. 샘플링/다양성 제어
- **subsample (기본=1.0)**  
  - 각 트리 학습에 사용할 샘플(행) 비율.  
  - 일반적으로 0.6~0.9 → 트리 다양성↑, 과적합↓.

- **colsample_bytree (기본=1.0)**  
  - 각 트리에서 사용할 특징(열) 비율.  
  - 0.5~0.9로 조정해 일반화 향상.

- **colsample_bylevel / colsample_bynode (기본=1.0)**  
  - 레벨/노드 단위에서 추가 특징 샘플링.  
  - 미세한 규제 효과.

---

## 📌 4. 정규화/규제
- **reg_lambda (lambda, 기본=1.0)**  
  - L2 정규화(가중치 크기 억제). 값↑ → 안정적 모델, 과적합 억제.

- **reg_alpha (alpha, 기본=0.0)**  
  - L1 정규화(가중치 희소화). 값↑ → 특징 선택 효과.  
  - 고차원 희소 데이터(예: 텍스트 분류)에 유용.

---

## 📌 5. 불균형 처리
- **scale_pos_weight (기본=1)**  
  - 양성/음성 클래스 불균형 보정.  
  - 대략 `neg/pos` 비율로 설정. (예: 음성 90%, 양성 10% → 9)

---

## 📌 6. 실행 최적화
- **tree_method**  
  - `"auto"`: 자동 선택  
  - `"hist"`: 대규모 데이터 빠른 학습  
  - `"gpu_hist"`: GPU 가속 → 대규모/고차원 데이터에서 권장

- **max_bin (기본=256)**  
  - 히스토그램 버킷 수. 값↑ → 정확도↑, 메모리·시간↑.

- **n_jobs (기본=None)**  
  - 병렬 처리 스레드 수. `-1` → 모든 코어 사용.

---

## 🚦 하이퍼파라미터 튜닝 우선순위 가이드

### ✅ 1순위 (모델 성능에 가장 큰 영향)
- **learning_rate** ↔ **n_estimators**  
- **max_depth**  
- **min_child_weight**

→ 트리 복잡도와 학습률/트리 개수의 조합이 성능에 직접적으로 영향.

---

### ⚖️ 2순위 (과적합 방지 및 일반화 성능 개선)
- **gamma**  
- **subsample**  
- **colsample_bytree**

→ 분할 억제 및 샘플링 다양성을 통해 오버핏 줄이고 안정적 일반화 유도.

---

### 🔧 3순위 (세밀한 조정/특수 상황)
- **reg_lambda**, **reg_alpha** → 정규화 강화  
- **scale_pos_weight** → 불균형 데이터 개선  
- **tree_method**, **max_bin** → 속도·메모리 최적화  

---

## 📌 요약
- **먼저 조정**: `learning_rate`, `n_estimators`, `max_depth`, `min_child_weight`  
- **그 다음**: `gamma`, `subsample`, `colsample_bytree`  
- **마지막 미세 조정**: `reg_lambda`, `reg_alpha`, `scale_pos_weight`  

이 순서대로 탐색하면, 효율적으로 성능을 개선할 수 있습니다.

## 3) 주요 속성 (Attributes)

- **feature_importances_**: 학습된 모델의 특징 중요도
- **best_score**, **best_iteration**, **best_ntree_limit**: 조기 종료(early stopping) 시 최적 결과
- **evals_result()**: 학습 중 평가 지표 기록
- **classes_** *(분류)*: 학습된 클래스 라벨 목록
- **n_features_in_**: 입력 특징 수

---

## 4) 주요 메서드 (Methods)

- **fit(X, y[, eval_set, early_stopping_rounds])**  
  모델 학습. `eval_set`과 `early_stopping_rounds`로 조기 종료 가능.
  - **eval_set** : list of (X, y), optional  
    - 검증 데이터셋 지정  
    - 예: `[(X_train, y_train), (X_valid, y_valid)]`  
    - 학습 로그와 조기 종료에 사용  
  - **verbose** : bool or int, default=None  
    - 학습 중 로그 출력 여부/빈도  
    - `verbose=True` → 각 라운드 성능 출력  
    - `verbose=10` → 10 라운드마다 출력  
- **predict(X)**  
  분류: 클래스 라벨, 회귀: 예측 값 반환.

- **predict_proba(X)** *(분류)*  
  클래스별 확률 예측.

- **score(X, y)**  
  기본은 정확도(분류) / R²(회귀).

- **get_booster()**  
  내부 Booster 객체 반환 (저수준 API 접근).

- **get_params() / set_params()**  
  하이퍼파라미터 조회/설정.

- **save_model(fname) / load_model(fname)**  
  모델 저장/불러오기 (JSON, binary).

- **evals_result()**  
  학습 중의 평가 지표 결과를 딕셔너리로 반환.

---

In [ ]:

import numpy as np
import pandas as pd

from xgboost import XGBClassifier, XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    classification_report, confusion_matrix, accuracy_score,
    r2_score, mean_absolute_error, mean_squared_error
)


In [7]:

# 파일 경로를 맞게 변경하세요.
body = pd.read_csv("./data/bodyPerformance.csv")
body.head(1)

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,27.0,M,172.3,75.24,21.3,80.0,130.0,54.9,18.4,60.0,217.0,C


In [21]:

# 타깃 인코딩 (A,B,C,D → 0,1,2,3)
class_map = {"A":0, "B":1, "C":2, "D":3}
body["target"] = body["class"].map(class_map)
body['gender'] = np.where(body['gender'] == 'M', 0, 1)
# 입력/타깃 분리
X_body = body.drop(columns=["class", "target"])
y_body = body["target"]


Xb_train, Xb_valid, yb_train, yb_valid = train_test_split(
    X_body, y_body, test_size=0.2, random_state=42, stratify=y_body
)

# XGBoost 분류기 (다중분류) — 조기 종료 사용
clf = XGBClassifier(
    objective="multi:softprob",   # 확률 출력
    eval_metric="mlogloss",
    n_estimators=1000,
    learning_rate=0.05,
    max_depth=5,
    min_child_weight=2,
    subsample=0.8,
    colsample_bytree=0.8,
    early_stopping_rounds=50,   
    reg_lambda=2.0,
    random_state=42,
    tree_method="hist"            # 대체: "gpu_hist" (GPU 사용 시)
)

clf.fit(
    Xb_train, yb_train, 
    eval_set=[(Xb_valid, yb_valid)], verbose=50
)

yb_pred = clf.predict(Xb_valid)
print("=== bodyPerformance: 다중 분류 ===")
print("Best iteration:", clf.best_iteration)
print("Accuracy:", accuracy_score(yb_valid, yb_pred))
print("\nClassification report:\n", classification_report(yb_valid, yb_pred, digits=4))
print("Confusion matrix:\n", confusion_matrix(yb_valid, yb_pred))


[0]	validation_0-mlogloss:1.35549
[50]	validation_0-mlogloss:0.83926
[100]	validation_0-mlogloss:0.72915
[150]	validation_0-mlogloss:0.68269
[200]	validation_0-mlogloss:0.65806
[250]	validation_0-mlogloss:0.64306
[300]	validation_0-mlogloss:0.63351
[350]	validation_0-mlogloss:0.62767
[400]	validation_0-mlogloss:0.62371
[450]	validation_0-mlogloss:0.62163
[500]	validation_0-mlogloss:0.61934
[550]	validation_0-mlogloss:0.61833
[600]	validation_0-mlogloss:0.61769
[650]	validation_0-mlogloss:0.61753
[680]	validation_0-mlogloss:0.61812
=== bodyPerformance: 다중 분류 ===
Best iteration: 630
Accuracy: 0.7521463232549459

Classification report:
               precision    recall  f1-score   support

           0     0.7193    0.8836    0.7930       670
           1     0.6331    0.6009    0.6166       669
           2     0.7444    0.6910    0.7167       670
           3     0.9316    0.8328    0.8794       670

    accuracy                         0.7521      2679
   macro avg     0.7571    0.752

In [20]:
car = pd.read_csv("./data/CarPrice_Assignment.csv")

car_num = car.select_dtypes(['number'])
features = list(car_num.columns.difference(['car_ID', 'symboling', 'price']))

x = car_num[features]
y = car_num['price']

## 데이터 분할 
from sklearn.model_selection import train_test_split

Xc_train, Xc_valid, yc_train, yc_valid = train_test_split(x, y, test_size=0.3, random_state=1)

# XGBoost 회귀기 — 조기 종료 사용
reg = XGBRegressor(
    objective="reg:squarederror",
    eval_metric="rmse",
    n_estimators=2000,
    learning_rate=0.03,
    max_depth=6,
    min_child_weight=3,
    subsample=0.8,
    colsample_bytree=0.8,
    reg_lambda=3.0,
    random_state=42,
    tree_method="hist",
    early_stopping_rounds=100
)

reg.fit(
    Xc_train, yc_train, eval_set=[(Xc_valid, yc_valid)], verbose=100
)

yc_pred = reg.predict(Xc_valid)
rmse = mean_squared_error(yc_valid, yc_pred)
mae = mean_absolute_error(yc_valid, yc_pred)
r2  = r2_score(yc_valid, yc_pred)

print("\n=== CarPrice_Assignment: 회귀 ===")
print("Best iteration:", reg.best_iteration)
print(f"RMSE: {rmse:.4f} | MAE: {mae:.4f} | R2: {r2:.4f}")

[0]	validation_0-rmse:7656.65072
[100]	validation_0-rmse:2394.08758
[200]	validation_0-rmse:2240.15496
[300]	validation_0-rmse:2197.80429
[400]	validation_0-rmse:2185.63579
[481]	validation_0-rmse:2191.59692

=== CarPrice_Assignment: 회귀 ===
Best iteration: 382
RMSE: 4765214.3643 | MAE: 1382.2057 | R2: 0.9211
